In [3]:
import importlib
import test_extract
import graph
importlib.reload(test_extract)
importlib.reload(graph)
import time
import matplotlib.pyplot as plt

In [26]:
start = time.time()
data = test_extract.Data("./data/CIS_enrollment.csv", ["Information Science"])
end = time.time()
print(end - start)

0.6604621410369873


# Data model test

In [27]:
len(data.courses),len(data.students)

(2372, 982)

In [28]:
print(list(data.students.values())[0])

{'id': '2109188', 'major': 'Information Science', 'courses': ['Computing Cultures', 'Physics II: Electromagnetism', 'Linear Algebra for Engineers', 'Earthquake!', 'Multivariable Calculus Engrs', 'Data Struct & Functional Progr', 'Intro Computing Using Python', 'Intro to Natural Lang Process', 'Networks II: Market Design', 'Teams and Technology', 'Data-Driven Web Applications', 'Probability Models & Inference', 'Systems Programming', 'Engineering Seminar', 'Programming Practicum', 'Operating Systems', 'Foundations of Artif Inllgnce', 'Cooperative Workshop MATH 1920', 'Prac in A I', 'Obj-Oriented Prog & Data Struc', 'Calculus For Engineers', 'Text Mining Hist & Literature', 'Networks', 'Engineering Communications', 'Information Retrieval', 'Cooperative Workshop CHEM 2090', 'Engineering General Chemistry', 'Physics I: Mechanics & Heat', 'Intro to Cognitive Science', 'Archery', 'Basic Engr Probability & Stats', 'Intro Analysis of Algorithms', 'Intro Microeconomics', 'Bowling', 'Discrete St

In [29]:
print(list(data.courses["CS1110"].students)[0])

{'id': '2317981', 'major': 'Information Science', 'courses': ['Beginning Swim', 'Marketing Principles', 'Obj-Oriented Prog & Data Struc', 'Communication and Technology', 'Intro Computing Using Python', 'Intro Design & Prog for Web', 'Introduction to Data Science', 'Data-Driven Web Applications', 'Elementary Hindi II', 'Probability Models & Inference', 'Systems Programming', 'Human Evolution', 'Info Ethics Law & Policy', 'Financial Accounting Principle', 'Networks', 'Programming Practicum', 'Human-Computer Interaction Des', 'The Language of Chemistry', 'Elementary Hindi I', 'Introductory Statistics', 'Designing Tech Social Impact', 'Linear Algebra', 'Beginning Tae Kwon Do', 'Poetry of Classical India', 'Inventing an Info Society'], 'terms': [('PE1100', (2014, 2)), ('CS2110', (2015, 0)), ('INFO2040', (2014, 2)), ('INFO3450', (2016, 2)), ('HINDI1101', (2015, 2)), ('INFO2950', (2016, 0)), ('CS3300', (2016, 0)), ('CS2111', (2015, 0)), ('ASIAN3366', (2016, 0)), ('INFO1200', (2017, 0)), ('ANT

# Graph model test

In [30]:
g = graph.Graph()

In [31]:
start = time.time()
c = 1
for cname1 in data.courses:
    if c % 250 == 0:
        print(c)
    for cname2 in data.courses:
        if cname2 == cname1:
            continue
        course = data.courses[cname1]
        count = 0
        for s in course.students:
            if cname2 in s.term_numbers and s.term_numbers[cname2] == s.term_numbers[cname1] + 1:
                count += 1
        if count > 0:
            g.addEdge(cname1, cname2, count)
    c += 1
end = time.time()
print(end - start)

250
500
750
1000
1250
1500
1750
2000
2250
11.621638059616089


In [34]:
for e in g.filterEdges(lambda e: e.weight > 75):
    print(str(e))

(INFO2040,INFO2300,230)
(INFO2040,CS2110,76)
(INFO2040,INFO2950,130)
(INFO2040,INFO1200,129)
(CS1110,INFO2040,131)
(CS1110,CS2800,91)
(CS1110,CS2111,114)
(CS1110,CS2110,318)
(CS1110,INFO1300,127)
(INFO2300,INFO2040,88)
(INFO2300,INFO2450,125)
(INFO2300,INFO3450,116)
(INFO2300,INFO4240,120)
(INFO2450,INFO2300,175)
(INFO2450,INFO2950,105)
(INFO2450,INFO1200,108)
(CS2110,INFO2040,116)
(CS2110,CS2800,98)
(CS2110,INFO2450,96)
(CS2110,INFO1300,101)
(INFO2950,INFO2450,81)
(INFO2950,INFO3450,76)
(INFO2950,INFO4240,76)
(INFO1300,INFO2300,344)
(INFO1300,CS2110,78)
(INFO1300,INFO2950,123)
(INFO1300,INFO1200,124)
(INFO1200,INFO2040,80)
(INFO1200,INFO2450,115)
(INFO1200,INFO3450,77)
(INFO1200,INFO4240,154)
(INFO1200,INFO1300,81)


In [37]:
g.export_graph("test3",75)

done! exported as test3.dot


In [36]:
x = [min(e.weight, 100) for e in g.edges]
num_bins = 25
n, bins, patches = plt.hist(x, num_bins, facecolor='blue', alpha=0.5)
plt.xlabel("edge weight")
plt.ylabel("frequency")
plt.title("distribution of edge weights (capped at 100)")
plt.show()

AttributeError: 'tuple' object has no attribute 'weight'

In [121]:
len(g.edges)

159731

In [134]:
edges_dict = {}
for e in g.edges:
    edges_dict[(e.src, e.dest)] = e.weight
print(len(edges_dict))

159731


In [135]:
for k in list(edges_dict.keys()):
    k1, k2 = k
    if k in edges_dict and (k2,k1) in edges_dict:
        w1, w2 = edges_dict[k], edges_dict[(k2, k1)]
        if w1/w2 > 4:
            del edges_dict[(k2, k1)]
        elif w2/w1 > 4:
            del edges_dict[k]
print(len(edges_dict))

156744


In [139]:
for k in list(edges_dict.keys()):
    if edges_dict[k] < 5:
        del edges_dict[k]
print(len(edges_dict))    

14225


In [141]:
n = set()
for k in list(edges_dict.keys()):
    n.add(k[0])
    n.add(k[1])
print(len(n))

922


In [142]:
len(g.nodes)

3625

In [146]:
def export_edges(edges, name:str):
    nodes = set([e.src for e in edges]).intersection(set([e.dest for e in edges]))
    fname = name+".dot"
    with open(fname, "w") as f:
        f.write("digraph course_graph {\n")
        for e in edges:
            f.write('{} -> {} [ label = "{}" ];\n'.format(e.src, e.dest, e.weight))
        for n in nodes:
            f.write('{} [ label = "{}" ];\n'.format(n,n))
        f.write("}\n")
    print("done! exported as {}.dot".format(name))

In [149]:
new_edges = [e for e in g.edges if (e.src, e.dest) in edges_dict and e.weight > 250]
export_edges(new_edges, "test2")

done! exported as test2.dot
